In [1]:
import logging
import sys

fx_spot_tickers = [
    "EURUSD=X",
    "GBPUSD=X",
    "JPYUSD=X",
    "AUDUSD=X",
    "NZDUSD=X",
    "USDCHF=X",
    "USDCAD=X",
    "USDSEK=X",
    "USDNOK=X",
]
logging.basicConfig(
    level=logging.INFO,
    stream=sys.stdout,
    format="[%(levelname)s] %(name)s: %(message)s",
    force=True,
)

In [2]:
import yfinance as yf
from importlib.resources import files
import backtest_lib.examples
import pickle as pkl

data = yf.download(
    tickers=fx_spot_tickers,
    start="2020-01-01",
    end="2025-01-01",
    interval="1d",
    auto_adjust=True,
)

pkl.dump(data, files(backtest_lib.examples).joinpath("fx_spot.pkl").open("wb"))

[*********************100%***********************]  9 of 9 completed


In [3]:
import backtest_lib as btl

px_flat = data["Close"].rename_axis(index="date", columns=None).ffill().reset_index()

market_view = btl.MarketView(prices=px_flat)

In [4]:
universe = tuple(fx_spot_tickers)


def index_strategy(universe, current_portfolio, market, ctx):
    return btl.Decision(current_portfolio)

In [5]:
from backtest_lib.portfolio import uniform_portfolio
from backtest_lib.backtest import Backtest

backtest = Backtest(
    market_view=market_view,
    universe=universe,
    strategy=index_strategy,
    initial_portfolio=uniform_portfolio(universe),
)

In [6]:
results = backtest.run()

In [7]:
results

BacktestResults(initial_capital=1.0, portfolio_returns=[0.0, 0.006426353015120682, -0.0012183887132028322, -0.0002647934952504428, -0.00039931328565146265, -0.0002839524412498885, -0.0002659405509797541, -0.0010891077549663774, 0.0014228886685247496, -0.0017630250105327536, -0.0005110824595689118, -0.00023023362408643687, 0.0009922384164432817, -0.000404111880448505, -0.00012870216460712482, 0.0008387055981693933, 0.0011364468221477136, 0.0005371818552093036, 0.0005055182129418456, 0.0005711244869196677, -0.0009479238112341794, 0.0007089203674570681, -0.00017610911683598092, 0.0009143202672783042, -0.00015447889797152328, -0.0009051139576148405, -0.0005867805395604471, -0.0007724871602541915, -0.00033759327982859375, -0.00022294678544693494, -0.00024337698330771694, 0.00010648785171035152, 0.0013759471996021258, 0.0005645045954751061, -0.0010539326790171642, 0.0009508159344428831, -0.0030914500422307813, -0.0018025165782024938, -0.00011815917273768832, 0.0014853437709383887, 0.00054916